

Grade cardinality thresholds: $m$ inequality constraints for the upper threshold of $N_j$



In [1]:
import numpy as np
import math
import time
import itertools
import numpy as np
#from scipy.sparse import csr_matrix #could be useful for treating the Q matrix in the multipl Y^T Q Y
n=4;
m=3;
maxx = 2 #upper threshold (lamda_2)
N_S2 = math.floor(1+math.log2(maxx))
mu2 = 90
print("in the code n does not represent the num of count. but the number of rows")
print("number of rows: ",n)
print("number of columns: ",m)
print("upper threshold for N_j: ",maxx)
print("number of slack vars per each grade: ",N_S2)
print("number of slack vars: ",N_S2*m)

#################################  nm  #       N_S2 * m      # tot
# n=4; m=3; maxx=2; #############  12  +        6             = 18    (262144 iterations)
y =                                n*m   	+   N_S2  * m
print("number of binary variables: ",y)
print("mu2: ",mu2)

in the code n does not represent the num of count. but the number of rows
number of rows:  4
number of columns:  3
upper threshold for N_j:  2
number of slack vars per each grade:  2
number of slack vars:  6
number of binary variables:  18
mu2:  90


Constraints, equivalent equations, cost function

\begin{align}
		\sum_{i}x_{ij} \le \lambda_2
\end{align}
\begin{align}
    \Phi_{jS}^{(42)} &= λ_2 - \sum_{i} x_{ij}- S_{j}^{(42)}
\end{align}
\begin{align}
	P^{(42)} _S (\vec{x},\vec{s}_{42})
	=
	\mu_{42S} \cdot \biggl( &
	\lambda_2 ^2 m +
	\sum_{(u_1,u_2) \in \overline{U}_2} x_{u_1} x_{u_2}
	+
	\sum_{(v_1,v_2) \in \overline{\overline{V}}_2 }  2^{\left \lfloor (v_1 +1)/m \right \rfloor + \left \lfloor (v_2 +1)/m \right \rfloor}
	s_{v_1}^{(42)} s_{v_2}^{(42)}\nonumber\\
	&+
	\sum_{u \in \overline{U}_1} (-2) \lambda_1 x_{u}+
	\sum_{v \in \overline{\overline{V}}_1 }(-1)\lambda_1 2^{\left \lfloor (v +1)/m \right \rfloor+1} s_{v}^{(42)}
	\nonumber\\
	&+
	\sum_{ (w_1,w_2) \in \overline{\overline{W}}_2 } 2^{\left \lfloor w_2 +1 /m \right \rfloor+1} x_{w_1} s_{w_2}^{(42)}
	\biggr).
\end{align}
where
\begin{align}
	\overline{U}_2 &= \{ (u_1,u_2) \in \mathbb{N}[1,nm]\times \mathbb{N}[1,nm]|
	u_1 = (i_1-1)m+j-1, u_2 = (i_2-1)m+j-1\}, \nonumber\\
	\overline{\overline{V}}_2&= \{ (v_1,v_2) \in \mathbb{N}[1,N_{S}^{(42)}m]\times \mathbb{N}[1,N_{S}^{(42)}m]|
	v_1= l_1 m+j-1, v_2 = l_2 m+j-1\},\nonumber\\
	\overline{U}_1 &= \{ u \in  \mathbb{N}[1,nm]|u = (i-1)m+j-1\},\nonumber\\
	\overline{\overline{V}}_1&= \{ v \in \mathbb{N}[1,N_{S}^{(42)}m]|v = lm+j -1\},\nonumber\\
	\overline{\overline{W}}_2&= \{ (w_1,w_2) \in \mathbb{N}[1,nm] \times \mathbb{N}[1,N_{S}^{(42)}m]| w_1 = (i-1) +j-1, w_2 = lm+j-1\}.
\end{align}


In [2]:
############################ U2
i2j1 = []; u2 = []
for i1 in range(n):
	for i2 in range(n):
		for j in range(m):
			i2j1.append([i1+1,i2+1,j+1]) #1-based indices
for item in i2j1:
	u_1=(item[0]-1)*m+item[2]
	u_2=(item[1]-1)*m+item[2]
	u2.append([u_1 - 1,u_2 - 1]) #0-based indices
print(u2)
############################ V2
l2j1 = []; v2 = []
for l1 in range(N_S2):
	for l2 in range(N_S2):
		for j in range(m):
			l2j1.append([l1,l2,j+1]) #1-based indices
for item in l2j1:
	ulj_1=(item[0])*m+item[2]
	ulj_2=(item[1])*m+item[2]
	v2.append([ulj_1 -1 ,ulj_2 -1]) #0-based indices
print(v2)
############################ U1
i1j1 = []; u = []
for i in range(n):
		for j in range(m):
			i1j1.append([i+1,j+1])#1-based indices
for item in i1j1:
	u.append((item[0]-1)*m+item[1]-1)#0-based indices
print(u)
############################ V1
lj_1 = []; v = [];
for l in range(N_S2):
		for j in range(m):
			lj_1.append([l,j+1])#1-based indices
for item in lj_1:
	v.append(item[0]*m+item[1]-1) #0-based indices
print(v)
############################ W2
lj_1 = []; w2 = [];
for i in range(n):
	for l in range(N_S2):
		for j in range(m):
			lj_1.append([i+1,l,j+1])#1-based indices
for item in lj_1:
	w2.append([(item[0]-1)*m+item[2]-1,item[1]*m+item[2]-1]) #0-based indices
print(w2)

[[0, 0], [1, 1], [2, 2], [0, 3], [1, 4], [2, 5], [0, 6], [1, 7], [2, 8], [0, 9], [1, 10], [2, 11], [3, 0], [4, 1], [5, 2], [3, 3], [4, 4], [5, 5], [3, 6], [4, 7], [5, 8], [3, 9], [4, 10], [5, 11], [6, 0], [7, 1], [8, 2], [6, 3], [7, 4], [8, 5], [6, 6], [7, 7], [8, 8], [6, 9], [7, 10], [8, 11], [9, 0], [10, 1], [11, 2], [9, 3], [10, 4], [11, 5], [9, 6], [10, 7], [11, 8], [9, 9], [10, 10], [11, 11]]
[[0, 0], [1, 1], [2, 2], [0, 3], [1, 4], [2, 5], [3, 0], [4, 1], [5, 2], [3, 3], [4, 4], [5, 5]]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[0, 1, 2, 3, 4, 5]
[[0, 0], [1, 1], [2, 2], [0, 3], [1, 4], [2, 5], [3, 0], [4, 1], [5, 2], [3, 3], [4, 4], [5, 5], [6, 0], [7, 1], [8, 2], [6, 3], [7, 4], [8, 5], [9, 0], [10, 1], [11, 2], [9, 3], [10, 4], [11, 5]]


In [3]:
# def func1(v1,v2):
#   return math.pow(2,math.floor((v1-n*m+1)/m)+math.floor((v2-n*m+1)/m))
# def func2(v):
#   return math.pow(2,1+math.floor(( v-n*m+1)/m))
def func3(v1,v2):
  return math.pow(2,math.floor((v1+1)/m)+math.floor((v2+1)/m))
def func4(v):
  return math.pow(2,1+math.floor(( v+1)/m))

In [4]:
offset = n*m
c=m*maxx*maxx
Q = np.zeros([y,y]) #y =    n*m   	+   N_S2  * m
for u_ind in u2:
  if u[0]==u[1]:
    Q[u_ind[0]][u_ind[1]] += 1
  else:
    Q[u_ind[0]][u_ind[1]] += 0.5
    Q[u_ind[1]][u_ind[0]] += 0.5

for v_ind in v2:
  if v_ind[0]==v_ind[1]:
    Q[offset+v_ind[0]][offset+v_ind[1]] += func3(v_ind[0],v_ind[1])
  else:
    Q[offset+v_ind[0]][offset+v_ind[1]] += 0.5*func3(v_ind[0],v_ind[1])
    Q[offset+v_ind[1]][offset+v_ind[0]] += 0.5*func3(v_ind[0],v_ind[1])

for u_ind in u:
    Q[u_ind,u_ind] += -2*maxx

for ind,v_inv in enumerate(v):
    Q[offset+ind][offset+ind] += -maxx*func4(v_inv)

for w_ind in w2:
    Q[w_ind[0]][offset+w_ind[1]] += 0.5*func4(w_ind[1])
    Q[offset+w_ind[1]][w_ind[0]] += 0.5*func4(w_ind[1])
c=mu2*c
Q=mu2*Q

print(Q)

[[-270.    0.    0.   90.    0.    0.   90.    0.    0.   90.    0.    0.
    90.    0.    0.  180.    0.    0.]
 [   0. -270.    0.    0.   90.    0.    0.   90.    0.    0.   90.    0.
     0.   90.    0.    0.  180.    0.]
 [   0.    0. -270.    0.    0.   90.    0.    0.   90.    0.    0.   90.
     0.    0.  180.    0.    0.  360.]
 [  90.    0.    0. -270.    0.    0.   90.    0.    0.   90.    0.    0.
    90.    0.    0.  180.    0.    0.]
 [   0.   90.    0.    0. -270.    0.    0.   90.    0.    0.   90.    0.
     0.   90.    0.    0.  180.    0.]
 [   0.    0.   90.    0.    0. -270.    0.    0.   90.    0.    0.   90.
     0.    0.  180.    0.    0.  360.]
 [  90.    0.    0.   90.    0.    0. -270.    0.    0.   90.    0.    0.
    90.    0.    0.  180.    0.    0.]
 [   0.   90.    0.    0.   90.    0.    0. -270.    0.    0.   90.    0.
     0.   90.    0.    0.  180.    0.]
 [   0.    0.   90.    0.    0.   90.    0.    0. -270.    0.    0.   90.
     0.    0.  180.   

Evaluation of all the values of the non-homogeneous part of the cost function $P^{(42)} _S (\vec{x},\vec{s}_{42})$

In [5]:
start_time = time.perf_counter()

Cy_vect = []
for i,item in enumerate(itertools.product([0, 1], repeat=y)):
  Y = np.array(item)
  #if (Y.dot(Q)).dot(Y.transpose()) != np.einsum('i,ij,j->', Y, Q, Y):
  #  print("wrong")
  Cy=np.einsum('i,ij,j->', Y, Q, Y, optimize=False) #the best choice is optimize=False
  Cy_vect.append(float(Cy))
print("is the minimum of the non-homogeneous part of the cost function")
print("equal to the costant c (homogeneous part)?")
if min(Cy_vect)+c==0:
  print("Yes")

end_time = time.perf_counter()
print(f"Time taken: {end_time - start_time} seconds")

is the minimum of the non-homogeneous part of the cost function
equal to the costant c (homogeneous part)?
Yes
Time taken: 4.1824247780000405 seconds


In [8]:
# this part checks if the preceding code provides results having N_j > maxx.
# if it happens then "wrong" is displayed
Y_vect = []
for ind,item in enumerate(itertools.product([0, 1], repeat=y)):
  if Cy_vect[ind]==-c:
    Y_vect.append(np.array(item)[:n*m].reshape(n, m))
    print(np.array(item)[:n*m].reshape(n, m))
for item_1 in Y_vect:
  for item_2 in np.sum(item_1,axis=0):
      if item_2 > maxx:
        print("wrong")

[[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 0 0]]
[[0 0 0]
 [0 0 0]
 [0 0 0]
 [0 1 0]]
[[0 0 0]
 [0 0 0]
 [0 0 0]
 [1 0 0]]
[[0 0 0]
 [0 0 0]
 [0 0 0]
 [1 1 0]]
[[0 0 0]
 [0 0 0]
 [0 0 1]
 [0 0 1]]
[[0 0 0]
 [0 0 0]
 [0 0 1]
 [0 1 1]]
[[0 0 0]
 [0 0 0]
 [0 0 1]
 [1 0 1]]
[[0 0 0]
 [0 0 0]
 [0 0 1]
 [1 1 1]]
[[0 0 0]
 [0 0 0]
 [0 1 0]
 [0 0 0]]
[[0 0 0]
 [0 0 0]
 [0 1 0]
 [0 1 0]]
[[0 0 0]
 [0 0 0]
 [0 1 0]
 [1 0 0]]
[[0 0 0]
 [0 0 0]
 [0 1 0]
 [1 1 0]]
[[0 0 0]
 [0 0 0]
 [0 1 1]
 [0 0 1]]
[[0 0 0]
 [0 0 0]
 [0 1 1]
 [0 1 1]]
[[0 0 0]
 [0 0 0]
 [0 1 1]
 [1 0 1]]
[[0 0 0]
 [0 0 0]
 [0 1 1]
 [1 1 1]]
[[0 0 0]
 [0 0 0]
 [1 0 0]
 [0 0 0]]
[[0 0 0]
 [0 0 0]
 [1 0 0]
 [0 1 0]]
[[0 0 0]
 [0 0 0]
 [1 0 0]
 [1 0 0]]
[[0 0 0]
 [0 0 0]
 [1 0 0]
 [1 1 0]]
[[0 0 0]
 [0 0 0]
 [1 0 1]
 [0 0 1]]
[[0 0 0]
 [0 0 0]
 [1 0 1]
 [0 1 1]]
[[0 0 0]
 [0 0 0]
 [1 0 1]
 [1 0 1]]
[[0 0 0]
 [0 0 0]
 [1 0 1]
 [1 1 1]]
[[0 0 0]
 [0 0 0]
 [1 1 0]
 [0 0 0]]
[[0 0 0]
 [0 0 0]
 [1 1 0]
 [0 1 0]]
[[0 0 0]
 [0 0 0]
 [1 1 0]
 [1 0 0]]
[

In [7]:
# we could use np.where(Cy_vect == np.min(Cy_vect))[0] to find the
# index corresponding to the minima of a vector